# Stats in R2lab

## Scope

The working assumption here is that the interesting data is

* the percentage of usage on a given period start .. end

* the total number of relevant accounts and slices

* also we might wish to pinpoint entries in the db that
  correspond to some operation conditions. For example,
  as of march 2018, we are interested in the disabled accounts
  attached to the r2lab site, because we suspect some people in
  this set have been trying to join but that was never acted upon...

## Changelog

* 2018 March; this a rebuild - see `stats-old.py` - of a previously rather ad hoc script. The present version will be OK for mostly 2017 and later, as we ignore the old data stored in json files.

* 2017 November; at that time - again, see `stats-old.py` - we used 2 different sources of data, presumably because of the migration from the omf/rest API to myplc.

# Convenience

This cell is only here so that any change in the code gets reloaded.

In [1]:
%load_ext autoreload
%autoreload 2

# Proxying / password to the API

You need to know the password of the PLCAPI admin account on `r2labapi.inria.fr`

In [2]:
import getpass
account = "root@r2lab.inria.fr"
password = getpass.getpass(f"Enter password for {account} : ")

Enter password for root@r2lab.inria.fr : ········


In [3]:
# let's check that
auth = {'AuthMethod' : 'password',
        'Username' : account,
        'AuthString' : password}

import xmlrpc.client
url = "https://r2labapi.inria.fr:443/PLCAPI/"

In [4]:
# if everything is OK this should print '1'
proxy = xmlrpc.client.ServerProxy(url)
try:
    print(proxy.AuthCheck(auth))
except Exception as e:
    print(f"OOPS, something wrong with {type(e)} - {e}")

1


*********

## Time formats

In [5]:
from timeutils import parse_date, human_readable

In [6]:
p1 = parse_date("2017-01-01")
p1

input_string 2017-01-01


1483225200.0

In [7]:
p2 = parse_date("2017-01-01", end=True)
p2

input_string 2017-01-01


1483311600.0

In [8]:
human_readable(p1)

'2016-12-31T23:00:UTC'

In [9]:
human_readable(p2)

'2017-01-01T23:00:UTC'

## Enter your period of interest

Second argument to `input_date` is the default answer

In [10]:
from timeutils import input_default

In [11]:
sfrom = input_default("Enter starting day", "2017-01-01")

Enter starting day [2017-01-01] : 


In [12]:
suntil = input_default("Enter ending day", "2017-12-31")

Enter ending day [2017-12-31] : 


In [13]:
ifrom, iuntil = parse_date(sfrom), parse_date(suntil, end=True)
print(f"Period of interest: {human_readable(ifrom)} - {human_readable(iuntil)}")
print(f"Total duration = {(iuntil-ifrom)//(24*3600)} days")

input_string 2017-01-01
input_string 2017-12-31
Period of interest: 2016-12-31T23:00:UTC - 2017-12-31T23:00:UTC
Total duration = 365.0 days


# Testbed busy ratio

## Fetching leases

In [14]:
# fetch leases for that period
selected_leases = proxy.GetLeases(
    auth,
    {'>t_from' : ifrom, '<t_from' : iuntil}
)
print(f"there have been {len(selected_leases)} reservations made during the period")

there have been 616 reservations made during the period


### Sorting in ascending order

In [15]:
selected_leases.sort(key=lambda lease: lease['t_from'])

### A glimpse

In [16]:
def oneliner(lease):
    return f"{lease['name']:25s} {human_readable(lease['t_from'])} -> {human_readable(lease['t_until'])}"

In [17]:
def glimpse(leases, size=5):
    for lease in leases[:size]:
        print(oneliner(lease))
    print("...")
    for lease in leases[-size:]:
        print(oneliner(lease))        

In [18]:
glimpse(selected_leases)

inria_r2lab.nightly       2017-01-06T15:00:UTC -> 2017-01-06T16:00:UTC
inria_naoufal.mesh        2017-01-09T15:30:UTC -> 2017-01-09T16:00:UTC
inria_r2lab.nightly       2017-01-09T16:00:UTC -> 2017-01-09T17:00:UTC
inria_r2lab.nightly       2017-01-11T02:00:UTC -> 2017-01-11T03:00:UTC
inria_r2lab.nightly       2017-01-12T02:00:UTC -> 2017-01-12T03:00:UTC
...
inria_r2lab.nightly       2017-12-27T02:00:UTC -> 2017-12-27T03:00:UTC
inria_pfe                 2017-12-27T14:40:UTC -> 2017-12-27T17:00:UTC
inria_pfe                 2017-12-27T23:00:UTC -> 2017-12-28T03:00:UTC
inria_cefore              2017-12-28T07:10:UTC -> 2017-12-28T08:00:UTC
inria_r2lab.nightly       2017-12-31T02:00:UTC -> 2017-12-31T03:00:UTC


## Usage ratio

##### raw ratio *vs* opening hours

The raw ratio is obtained by comparing the amount of time reserved with the total amount of time available.

Assuming that opening hours would be mon-fri from 09:00 to 19:00

In [19]:
open_correction = (5 * 10) / (7 * 24)
print(f"Opening hours are {open_correction:.2%} of total hours")

Opening hours are 29.76% of total hours


##### user *vs* admin

We try to classify the various slices in 2 families whether they are for management/operations purposes, or used for actual experimentation.

In [20]:
total_duration = iuntil - ifrom

In [21]:
def show_usage_ratio(leases, total_duration, message):

    def duration(lease):
        return lease['t_until'] - lease['t_from']

    reserved_duration = sum(duration(lease) for lease in leases)
    print(f"Total time reserved: {reserved_duration} / {total_duration} s")
    print(f"                i.e: {reserved_duration/3600:.2f} / {total_duration/3600:.2f}    hours")
    print(f"                i.e: {reserved_duration/(24*3600):.2f} / {total_duration/(24*3600):.2f}       days")
    
    raw_ratio = reserved_duration / total_duration
    print(f"{message}: raw_ratio is {raw_ratio:.2%}")
    
    open_ratio = raw_ratio / open_correction
    print(f"{message}: open_ratio is {open_ratio:.2%}")

In [22]:
show_usage_ratio(selected_leases, total_duration, "ALL LEASES")

Total time reserved: 4041600 / 31536000.0 s
                i.e: 1122.67 / 8760.00    hours
                i.e: 46.78 / 365.00       days
ALL LEASES: raw_ratio is 12.82%
ALL LEASES: open_ratio is 43.06%


### Usage ratio - filtered

We discard slices whose name contains any of the following

In [23]:
admin_slices = ['auto_', 'nightly', 'maintenance', 'tutorial']

In [24]:
def relevant(lease):
    return not any(admin in lease['name'] for admin in admin_slices)

filtered_leases = [lease for lease in selected_leases if relevant(lease)]

In [25]:
show_usage_ratio(filtered_leases, total_duration, "FILTERED LEASES")

Total time reserved: 3663000 / 31536000.0 s
                i.e: 1017.50 / 8760.00    hours
                i.e: 42.40 / 365.00       days
FILTERED LEASES: raw_ratio is 11.62%
FILTERED LEASES: open_ratio is 39.03%


## User accounts created

In [26]:
all_accounts = proxy.GetPersons(auth)

print(f"We have {len(all_accounts)} accounts")

We have 82 accounts


In [27]:
#all_accounts[0]

This will add a `login_base` field to each user account, to identify the attached institution:

In [28]:
all_sites = proxy.GetSites(auth)
site_hash = { site['site_id'] : site for site in all_sites}
for account in all_accounts:
    site_ids = account['site_ids']
    if not site_ids:
        account['login_base'] = '(none)'
    if len(site_ids) == 1:
        account['login_base'] = site_hash[account['site_ids'][0]]['login_base']
    else:
        account['login_base'] = '(' + ",".join(site_hash[site_id]['login_base'] for site_id in site_ids) + ')'

### detailed list

In [29]:
def show_accounts(accounts):
    accounts.sort(key = lambda person: person['date_created'])
    for i, account in enumerate(accounts, 1):
        print(f"[{i:02d}] "
              f" {'OK' if account['enabled'] else 'KO'}"
              f" {human_readable(account['date_created'])}"
              f" {account['login_base']:24s}"
              f" {account['email']}"
             )

#### all accounts

In [30]:
show_accounts(all_accounts)

[01]  OK 2016-12-16T13:27:UTC ()                       maint@localhost.localdomain
[02]  OK 2016-12-16T13:27:UTC auto                     root@r2lab.inria.fr
[03]  OK 2016-12-16T13:27:UTC inria                    anas.elankouri@gmail.com
[04]  OK 2016-12-16T13:27:UTC unicamp                  ramonreisfontes@gmail.com
[05]  OK 2016-12-16T13:27:UTC eurecomple               navid.nikaein@eurecom.fr
[06]  OK 2016-12-16T13:27:UTC upmc                     loic.baron@lip6.fr
[07]  OK 2016-12-16T13:27:UTC eurecomple               florian.kaltenberger@eurecom.fr
[08]  KO 2016-12-16T13:27:UTC inria                    mario-antonio.zancanaro@inria.fr
[09]  OK 2016-12-16T13:27:UTC inria                    mohamed-naoufal.mahfoudi@inria.fr
[10]  OK 2016-12-16T13:27:UTC inria                    frederic.saint-marcel@inria.fr
[11]  OK 2016-12-16T13:27:UTC inria                    thierry.parmentelat@inria.fr
[12]  OK 2016-12-16T13:27:UTC inria                    farzaneh.pazkad@inria.fr
[13]  OK 2016

### narrowing on the selected period

In [31]:
selected_accounts = [
    account for account in all_accounts 
    if account['date_created'] >= ifrom and
       account['date_created'] <= iuntil
]

show_accounts(selected_accounts)

[01]  KO 2017-01-10T11:52:UTC inria                    mario.zancanaro@inria.fr
[02]  OK 2017-01-11T11:37:UTC inria                    fabrice.valois@insa-lyon.fr
[03]  OK 2017-01-11T11:37:UTC inria                    jad.oueis@insa-lyon.fr
[04]  OK 2017-01-11T11:37:UTC inria                    razvan.stanica@inria.fr
[05]  OK 2017-01-24T16:22:UTC inria                    arnaud.legout@inria.fr
[06]  KO 2017-02-06T08:48:UTC r2lab                    raj85.vishwakarma@gmail.com
[07]  OK 2017-02-15T18:19:UTC inria                    mondi.ravi@inria.fr
[08]  OK 2017-02-15T18:25:UTC inria                    mondi.ravi@gmail.com
[09]  OK 2017-03-04T06:46:UTC inria                    kishoregolla9@gmail.com
[10]  KO 2017-03-09T09:12:UTC r2lab                    yassir.mr4@gmail.com
[11]  OK 2017-03-09T09:57:UTC inria                    yassir.mr1@gmail.com
[12]  OK 2017-03-09T19:18:UTC inria                    sebastian.fehlandt@inria.cl
[13]  OK 2017-04-06T19:36:UTC inria                   

In [32]:
# show number of enabled accounts in this list
new_accounts_in_selected_period = [
    account for account in selected_accounts if account['enabled']
]
print(f"New accounts in the selected period = {len(new_accounts_in_selected_period)}")

New accounts in the selected period = 32


### validated *vs* non-validated accounts

Still on the selected period, show the ones that were enabled or not

In [33]:
enabled_selected_accounts = [ account for account in selected_accounts if account['enabled']]
disabled_selected_accounts = [ account for account in selected_accounts if not account['enabled']]

In [34]:
show_accounts(enabled_selected_accounts)

[01]  OK 2017-01-11T11:37:UTC inria                    fabrice.valois@insa-lyon.fr
[02]  OK 2017-01-11T11:37:UTC inria                    jad.oueis@insa-lyon.fr
[03]  OK 2017-01-11T11:37:UTC inria                    razvan.stanica@inria.fr
[04]  OK 2017-01-24T16:22:UTC inria                    arnaud.legout@inria.fr
[05]  OK 2017-02-15T18:19:UTC inria                    mondi.ravi@inria.fr
[06]  OK 2017-02-15T18:25:UTC inria                    mondi.ravi@gmail.com
[07]  OK 2017-03-04T06:46:UTC inria                    kishoregolla9@gmail.com
[08]  OK 2017-03-09T09:57:UTC inria                    yassir.mr1@gmail.com
[09]  OK 2017-03-09T19:18:UTC inria                    sebastian.fehlandt@inria.cl
[10]  OK 2017-04-06T19:36:UTC inria                    victor.codocedo@inria.cl
[11]  OK 2017-04-11T16:35:UTC inria                    osama.arouk@inria.fr
[12]  OK 2017-05-07T13:16:UTC inria                    cappuzzo@eurecom.fr
[13]  OK 2017-05-19T10:29:UTC (eurecomple,eurecoms3)   hayes@e

In [35]:
show_accounts(disabled_selected_accounts)

[01]  KO 2017-01-10T11:52:UTC inria                    mario.zancanaro@inria.fr
[02]  KO 2017-02-06T08:48:UTC r2lab                    raj85.vishwakarma@gmail.com
[03]  KO 2017-03-09T09:12:UTC r2lab                    yassir.mr4@gmail.com
[04]  KO 2017-05-05T12:27:UTC eurecomple               kharade.rohan@iitb.ac.in
[05]  KO 2017-05-11T09:48:UTC eurecomple               info@try-rx.com
[06]  KO 2017-05-19T20:18:UTC eurecomple               yoseph.maloche@studenti.unitn.it
[07]  KO 2017-05-24T08:09:UTC inria                    qwerty1643@skku.edu
[08]  KO 2017-06-27T09:29:UTC unicamp                  christophe.letoquin@orange.com
[09]  KO 2017-07-03T06:52:UTC inria                    manipooja1990@gmail.com
[10]  KO 2017-08-26T17:09:UTC r2lab                    rayansalhab@hotmail.com
[11]  KO 2017-08-27T07:31:UTC inria                    xo79@qq.com
[12]  KO 2017-09-04T11:53:UTC r2lab                    29776716@qq.com
[13]  KO 2017-09-12T14:57:UTC inria                    zgouaksvic

# Specific to 2018

## the ones who registered with the `r2lab` site

**NOT** restricted to the selected period

In [36]:
r2lab_selected_accounts = [ account for account in all_accounts if 'r2lab' in account['login_base']]

In [37]:
show_accounts(r2lab_selected_accounts)

[01]  KO 2017-02-06T08:48:UTC r2lab                    raj85.vishwakarma@gmail.com
[02]  KO 2017-03-09T09:12:UTC r2lab                    yassir.mr4@gmail.com
[03]  KO 2017-08-26T17:09:UTC r2lab                    rayansalhab@hotmail.com
[04]  KO 2017-09-04T11:53:UTC r2lab                    29776716@qq.com
[05]  KO 2017-09-27T18:42:UTC r2lab                    yuchia.tseng@irt-systemx.fr
[06]  KO 2017-10-01T00:36:UTC r2lab                    sample@email.tst
[07]  KO 2017-10-03T03:37:UTC r2lab                    a5566740293@gmail.com
[08]  KO 2017-10-17T21:09:UTC r2lab                    rayclaytonrudman@hotmail.com
[09]  KO 2017-10-28T13:32:UTC r2lab                    hishamalzain21@gmail.com
[10]  KO 2017-10-29T09:29:UTC r2lab                    hisham_alzain21@yahoo.com
[11]  KO 2017-11-04T09:10:UTC r2lab                    wuzhiguocarter@gmail.com
[12]  KO 2017-12-13T11:00:UTC r2lab                    ayoub.bousselmi@orange.com
[13]  KO 2018-01-11T14:56:UTC r2lab                 

## other disabled accounts

In [38]:
other_disabled_accounts = [ account for account in all_accounts 
                           if not account['enabled'] and 'r2lab' not in account['login_base']]

In [39]:
show_accounts(other_disabled_accounts)

[01]  KO 2016-12-16T13:27:UTC inria                    mario-antonio.zancanaro@inria.fr
[02]  KO 2017-01-10T11:52:UTC inria                    mario.zancanaro@inria.fr
[03]  KO 2017-05-05T12:27:UTC eurecomple               kharade.rohan@iitb.ac.in
[04]  KO 2017-05-11T09:48:UTC eurecomple               info@try-rx.com
[05]  KO 2017-05-19T20:18:UTC eurecomple               yoseph.maloche@studenti.unitn.it
[06]  KO 2017-05-24T08:09:UTC inria                    qwerty1643@skku.edu
[07]  KO 2017-06-27T09:29:UTC unicamp                  christophe.letoquin@orange.com
[08]  KO 2017-07-03T06:52:UTC inria                    manipooja1990@gmail.com
[09]  KO 2017-08-27T07:31:UTC inria                    xo79@qq.com
[10]  KO 2017-09-12T14:57:UTC inria                    zgouaksvic@mailstome.today
[11]  KO 2017-09-26T19:49:UTC eurecoms3                dzakwanfalih@hotmail.com
[12]  KO 2017-10-01T01:00:UTC eurecomple               j1ltnl01@r2labapi.pl.sophia.inria.fr
[13]  KO 2017-11-01T07:57:UTC un